In [2]:
import numpy as np
from get_data import create_directories, build_search_url, download_extended_page, download_image, get_all_items

In [8]:
f = open('people.txt')
keywords = f.read().splitlines()
# keywords = ['omar sy', 'elon musk']
print("Number of keywords:", len(keywords))

Number of keywords: 100


In [10]:
sufix = "face"

keywords = [keyword + " " + sufix for keyword in keywords]
search_keywords = [str(item) for item in keywords]

limit = 1400

main_directory = "data/cnn/download"

In [ ]:
i_search_keywoard = 0

while i_search_keywoard < len(search_keywords):
	search_term = search_keywords[i_search_keywoard]

	subdirectory_name = search_term
	create_directories(main_directory, subdirectory_name)

	url = build_search_url(search_term)
	raw_html = download_extended_page(url)

	get_all_items(raw_html, main_directory, subdirectory_name, limit)

	i_search_keywoard += 1